In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
from sklearn.datasets import load_files

train_data = pd.read_csv('data/train.csv', header = 0)

print("type of instagram train data: {}".format(type(train_data)))
print("length of instagram train data: {}".format(len(train_data)))
print("example line of instagram train data:\n")
print(train_data.iloc[0])

type of instagram train data: <class 'pandas.core.frame.DataFrame'>
length of instagram train data: 576
example line of instagram train data:

profile pic                1.00
nums/length username       0.27
fullname words             0.00
nums/length fullname       0.00
name==username             0.00
description length        53.00
external URL               0.00
private                    0.00
#posts                    32.00
#followers              1000.00
#follows                 955.00
fake                       0.00
Name: 0, dtype: float64


In [22]:
test_data = pd.read_csv('data/test.csv', header = 0)

print("type of instagram test data: {}".format(type(test_data)))
print("length of instagram test data: {}".format(len(test_data)))
print("example line of instagram test data:\n")
print(test_data.iloc[0])

type of instagram test data: <class 'pandas.core.frame.DataFrame'>
length of instagram test data: 120
example line of instagram test data:

profile pic               1.00
nums/length username      0.33
fullname words            1.00
nums/length fullname      0.33
name==username            1.00
description length       30.00
external URL              0.00
private                   1.00
#posts                   35.00
#followers              488.00
#follows                604.00
fake                      0.00
Name: 0, dtype: float64
